# Importing the dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install arrow keybert bertopic
!pip install scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from transformers import BertTokenizer
from arrow import now
from keybert import KeyBERT
from bertopic import BERTopic
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import json
from sklearn.model_selection import train_test_split

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
dataset_path = '/content/drive/My Drive/AI4Good Mental Health Project/Datasets/20200325_counsel_chat.csv'
counsel2020_df = pd.read_csv(dataset_path)
counsel2020_df.shape

(2129, 12)

In [4]:
counsel2020_df

,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train
1,1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train
2,2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train
3,3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train
4,4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train
...,...,...,...,...,...,...,...,...,...,...,...,...
2124,2124,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,Victoria HaagFind direction for your life...,https://counselchat.com/therapists/victoria-haag,There are probably no two therapists alike bec...,0,45,train
2125,2125,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,Allison VelezIs your relationship worth it?,https://counselchat.com/therapists/allison-velez,"Each counselor may have a different process, s...",0,37,train
2126,2126,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,David RouttPresident and Clinical Counselor at...,https://counselchat.com/therapists/david-routt,"After meeting a client, many Counselors will a...",0,56,train
2127,2127,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,2nd Chance Counseling Service Online Addiction...,https://counselchat.com/therapists/2nd-chance-...,A good therapist will discuss what brought you...,0,27,train


In [12]:
dataset_path = '/content/drive/My Drive/AI4Good Mental Health Project/Datasets/20220401_counsel_chat.csv'
counsel2022_df = pd.read_csv(dataset_path)
counsel2022_df.shape

(2775, 10)

In [ ]:
counsel2022_df

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256
...,...,...,...,...,...,...,...,...,...,...
2770,939,Are some clients more difficult than others?,What are some difficulties that a counselor ca...,https://counselchat.com/questions/are-some-cli...,counseling-fundamentals,Audrey ONealBi-lingual Psychotherapist and Hea...,https://counselchat.com/therapists/audrey-oneal-3,Although many clients have the capacity to be ...,0,47
2771,939,Are some clients more difficult than others?,What are some difficulties that a counselor ca...,https://counselchat.com/questions/are-some-cli...,counseling-fundamentals,"Kaileen McMickle, MS, LPCLicensed Professional...",https://counselchat.com/therapists/kaileen-mcm...,"I usually don't label a client as ""difficult"" ...",0,22
2772,939,Are some clients more difficult than others?,What are some difficulties that a counselor ca...,https://counselchat.com/questions/are-some-cli...,counseling-fundamentals,"Dr. Timothy Paul'man'; Online - ""Natural Healt...",https://counselchat.com/therapists/dr-timothy-...,"Dang right! :)Heh heh, and correct me if I'm ...",0,23
2773,939,Are some clients more difficult than others?,What are some difficulties that a counselor ca...,https://counselchat.com/questions/are-some-cli...,counseling-fundamentals,"David KleinHumanistic, LGBT-Affirmative Psycho...",https://counselchat.com/therapists/david-klein,"Yes, just like some relationships outside of o...",0,41


In [6]:
ali_dataset_df= pd.read_parquet('/content/drive/My Drive/AI4Good Mental Health Project/Datasets/ali_train.parquet')
# ali_dataset.to_csv('/content/drive/My Drive/AI4Good Mental Health Project/ali_train.csv')   # save .parquet file to .csv for reference
ali_dataset_df.shape

(175, 1)

In [ ]:
ali_dataset_df

,text
0,<HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1,<HUMAN>: What are symptoms of panic attack vs....
2,<HUMAN>: What are the types of Mental Illness?...
3,<HUMAN>: What does mental-illness mean?\n<ASSI...
4,<HUMAN>: How can you treat mental illness?\n<A...
...,...
170,<HUMAN>: Who is the best hypnotherapist in Lon...
171,<HUMAN>: Who is the best psychiatrist in Mumba...
172,<HUMAN>: What to do if the support does not he...
173,<HUMAN>: What to do if i'm worried about my me...


In [7]:
# please do ensure that the dataset do not publish in public
psych8k_df = pd.read_json('/content/drive/My Drive/AI4Good Mental Health Project/Datasets/psych8k.json')
psych8k_df.shape

(8187, 4)

In [ ]:
psych8k_df

,instruction,input,output,instructions
0,"If you are a counsellor, please answer the que...","Lately, I've been feeling really disconnected ...",Thank you for sharing your experience with me....,NaN
1,"If you are a counsellor, please answer the que...","Lately, I've been feeling really stressed and ...",Thank you for sharing your experience with me....,NaN
2,"If you are a counsellor, please answer the que...","Lately, I've been feeling a bit off. I sometim...",I appreciate you sharing your concerns with me...,NaN
3,"If you are a counsellor, please answer the que...","Lately, I've been feeling a bit off. I've noti...",Thank you for sharing your feelings and experi...,NaN
4,"If you are a counsellor, please answer the que...","Lately, I've been feeling quite off. I have be...",I appreciate you sharing your experiences with...,NaN
...,...,...,...,...
8182,"If you are a counsellor, please answer the que...",Sometimes I think we do that to kind of surviv...,I understand that it can be difficult to show ...,NaN
8183,"If you are a counsellor, please answer the que...",I'm constantly flip-flopping and feeling frust...,I hear that you are experiencing frustration a...,NaN
8184,"If you are a counsellor, please answer the que...","I am stressed about school, my friends, and th...",It sounds like you are facing a lot of pressur...,NaN
8185,"If you are a counsellor, please answer the que...",I feel like I'm stuck between two realities - ...,It sounds like you're struggling to find your ...,NaN


## Trigger words

     (not a part of the merged dataset)

In [8]:
Trigger_words_df = pd.read_csv('/content/drive/My Drive/AI4Good Mental Health Project/Datasets/Trigger_words.csv')
Trigger_words_df.shape

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/AI4Good Mental Health Project/Datasets/Trigger_words.csv'

In [ ]:
Trigger_words_df

,Words
0,*Ableism
1,*Abortion
2,*Abusive relationship
3,*Acephobia
4,*Ageism
...,...
283,carbon monoxide
284,poison myself
285,group suicides
286,mass suicides


# Merging all datasets

In [20]:
# Clean Ali's Dataset
def remove_newline_patterns(text):
    text = re.sub(r'\\n', '', text)
    text = re.sub(r'/n', '', text)
    return text

# split the <HUMAN> response and <ASSISTANT> response and create two separate columns named as "Input" & "Output"
def clean_split(row):
  row = remove_newline_patterns(row)
  parts = row.split("<ASSISTANT>:")

  if len(parts) < 2:
    return parts[0], ""
  else:
    input = parts[0].replace("<HUMAN>: ", "<HUMAN>:")
    input = parts[0].replace("<HUMAN>:", "").strip()
    output = parts[1].replace("<ASSISTANT>: ", "").strip()
    return input, output

split_df = ali_dataset_df['text'].apply(lambda x: clean_split(x)).apply(pd.Series)
ali_dataset_df[['Context', 'Response']] = split_df
ali_dataset_df.drop(columns=['text'], inplace=True)
ali_dataset_df

KeyError: 'text'

In [14]:
counsel2020_df['Context'] = counsel2020_df['questionTitle'] + ' ' + counsel2020_df['questionText']
counsel2020_df.rename(columns={'answerText': 'Response'}, inplace=True)
counsel2020_df = counsel2020_df[[ 'Context', 'Response']]
counsel2020_df

,Context,Response
0,Can I change my feeling of being worthless to ...,"If everyone thinks you're worthless, then mayb..."
1,Can I change my feeling of being worthless to ...,"Hello, and thank you for your question and see..."
2,Can I change my feeling of being worthless to ...,First thing I'd suggest is getting the sleep y...
3,Can I change my feeling of being worthless to ...,Therapy is essential for those that are feelin...
4,Can I change my feeling of being worthless to ...,I first want to let you know that you are not ...
...,...,...
2124,What happens in a counseling session? After fi...,There are probably no two therapists alike bec...
2125,What happens in a counseling session? After fi...,"Each counselor may have a different process, s..."
2126,What happens in a counseling session? After fi...,"After meeting a client, many Counselors will a..."
2127,What happens in a counseling session? After fi...,A good therapist will discuss what brought you...


In [19]:
# Concatenate title with text as context
counsel2022_df['Context'] = counsel2022_df['questionTitle'] + ' ' + counsel2022_df['questionText']
counsel2022_df.rename(columns={'answerText': 'Response'}, inplace=True)
counsel2022_df = counsel2022_df[[ 'Context', 'Response']]
counsel2022_df

KeyError: 'questionTitle'

In [17]:
psych8k_df.rename(columns={'input': 'Context', 'output': 'Response'}, inplace=True)
psych8k_df.drop(columns=['instruction', 'instructions'], inplace=True)
psych8k_df

,Context,Response
0,"Lately, I've been feeling really disconnected ...",Thank you for sharing your experience with me....
1,"Lately, I've been feeling really stressed and ...",Thank you for sharing your experience with me....
2,"Lately, I've been feeling a bit off. I sometim...",I appreciate you sharing your concerns with me...
3,"Lately, I've been feeling a bit off. I've noti...",Thank you for sharing your feelings and experi...
4,"Lately, I've been feeling quite off. I have be...",I appreciate you sharing your experiences with...
...,...,...
8182,Sometimes I think we do that to kind of surviv...,I understand that it can be difficult to show ...
8183,I'm constantly flip-flopping and feeling frust...,I hear that you are experiencing frustration a...
8184,"I am stressed about school, my friends, and th...",It sounds like you are facing a lot of pressur...
8185,I feel like I'm stuck between two realities - ...,It sounds like you're struggling to find your ...


In [21]:
merged_df = pd.concat([counsel2020_df, counsel2022_df, ali_dataset_df,psych8k_df],ignore_index=True)

In [25]:
merged_df.shape

(10979, 2)

# Handling missing values and duplicates

In [27]:
merged_df.dropna(inplace=True)
merged_df.drop_duplicates(inplace=True)
merged_df.shape

(10979, 2)

In [28]:
merged_df.to_csv('combined_dataframe.csv', index=False)
merged_df.head()

,Context,Response
0,Can I change my feeling of being worthless to ...,"If everyone thinks you're worthless, then mayb..."
1,Can I change my feeling of being worthless to ...,"Hello, and thank you for your question and see..."
2,Can I change my feeling of being worthless to ...,First thing I'd suggest is getting the sleep y...
3,Can I change my feeling of being worthless to ...,Therapy is essential for those that are feelin...
4,Can I change my feeling of being worthless to ...,I first want to let you know that you are not ...


# Convert csv to jsonl and split train-test

In [29]:
def csv_to_jsonl_split(csv_file_path, train_jsonl_file_path, test_jsonl_file_path):
    # Read the CSV file
    df = pd.read_csv(csv_file_path)

    # Set the test size and random state
    test_size = 0.2
    random_state = 42

    #========#
    data = []
    for index, row in df.iterrows():
        data.append({"prompt": row["Context"], "completion": row["Response"]})
        # this Chat Completion API format is when we use GPT 3.5

        # conversation = {
        # "messages": [
        #     {"role": "user", "content": row["Input"]},
        #     {"role": "assistant", "content": row["Response"]}
        # ]
        # }
        # data.append(conversation)

        # this is the format when we use GPT 4
    #========#

    # Split the dataframe into train and test sets
    train_df, test_df = train_test_split(data, test_size=test_size, random_state=random_state)

    # Function to write a dataframe to a JSONL file
    # def write_jsonl(df, jsonl_file_path):
    #     with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    #         for record in df.to_dict(orient='records'):
    #             jsonl_file.write(json.dumps(record) + '\n')

    # Function to write data to jsonl file
    def write_jsonl(data, jsonl_file_path):
        with open(jsonl_file_path, 'w', encoding='utf-8') as f:
            for item in data:
                f.write(json.dumps(item) + '\n')

    # Write the train and test data to JSONL files
    write_jsonl(train_df, train_jsonl_file_path)
    write_jsonl(test_df, test_jsonl_file_path)

csv_file_path = '/content/drive/My Drive/AI4Good Mental Health Project/Datasets/combined_dataframe.csv'
train_jsonl_file_path = '/content/drive/My Drive/AI4Good Mental Health Project/Datasets/train_data.jsonl'
test_jsonl_file_path = '/content/drive/My Drive/AI4Good Mental Health Project/Datasets/test_data.jsonl'
csv_to_jsonl_split(csv_file_path, train_jsonl_file_path, test_jsonl_file_path)

# Cleaning data

In [ ]:
def clean_data(text):
    tokens = word_tokenize(text)  # Tokenize
    tokens = [token.lower() for token in tokens]  # Lowercase
    tokens = [re.sub(r'[^a-zA-Z0-9. /,]', '', token) for token in tokens]  # Remove special characters
    tokens = [re.sub(r'http\S+', '', token) for token in tokens]  # Remove URLs
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]  # Remove punctuation
    tokens = [re.sub(r'\s+', ' ', token) for token in tokens]  # Remove extra spaces
    tokens = [token for token in tokens if token]  # Remove empty tokens
    tokens = [token.strip() for token in tokens]  # Remove leading and trailing spaces
    tokens = [token for token in tokens if token not in stopwords.words('english')]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatization
    return ' '.join(tokens)

merged_df['Clean_Context'] = merged_df['Context'].apply(clean_data)
merged_df['Clean_Response'] = merged_df['Response'].apply(clean_data)
merged_df.head()

,Context,Response,Clean_Context,Clean_Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",going thing feeling barely sleep nothing think...,everyone think worthless maybe need find new p...
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",going thing feeling barely sleep nothing think...,hello thank question seeking advice feeling wo...
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,going thing feeling barely sleep nothing think...,first thing suggest getting sleep need impact ...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,going thing feeling barely sleep nothing think...,therapy essential feeling depressed worthless ...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,going thing feeling barely sleep nothing think...,first want let know alone feeling always someo...


## `Trigger word cleaning`

In [ ]:
#Cleaning & handling missing values and duplicates for trigger
Trigger_words_df.dropna(inplace=True)
Trigger_words_df.drop_duplicates(inplace=True)
Trigger_words_df['Clean_Words'] = Trigger_words_df['Words'].apply(clean_data)
Trigger_words_df.head()

,Words,Clean_Words
0,*Ableism,ableism
1,*Abortion,abortion
2,*Abusive relationship,abusive relationship
3,*Acephobia,acephobia
4,*Ageism,ageism


In [ ]:
Clean_Trigger_words_df = Trigger_words_df.drop('Words', axis=1) #remove unnecessary columns
Clean_Trigger_words_df.head()
Clean_Trigger_words_df.to_csv('/content/drive/My Drive/AI4Good Mental Health Project/Datasets/Clean_Trigger_words.csv', index=False) #make a new cleaned CSV file

,Clean_Words
0,ableism
1,abortion
2,abusive relationship
3,acephobia
4,ageism


# Using BERTopic for Topic Modelling

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

# BERTopic Pipeline (we can modify the techniques):
#   Embedding Extraction:       SBERT
#   Dimensionality Reduction:   UMAP (optional: PCA)
#   Clustering:                 HDBCAN (optional: k-Means or other clustering methods)
#   Bag-of-words:               CountVectorizer
#   Topic Representation:       TD-IDF

# Initialize CountVectorizer with stop words removal (1 or 2 words)
vectorizer_model = CountVectorizer(stop_words='english', ngram_range=(1, 2))
sentence_model = SentenceTransformer("all-MiniLM-L12-v2")
topic_model = BERTopic(embedding_model=sentence_model,n_gram_range=(1, 2), vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(merged_df['Context'])

topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,4199,-1_like_feel_don_think,"[like, feel, don, think, want, feel like, just...",[I want a secure relationship with someone tha...
1,0,311,0_partner_relationship_like partner_mad,"[partner, relationship, like partner, mad, doe...",[That phrase makes me crazy. It happens anytim...
2,1,212,1_mood_energy_lately ve_ve feeling,"[mood, energy, lately ve, ve feeling, lately, ...","[Lately, I've been feeling pretty down, and it..."
3,2,199,2_counsellor_session_patient_therapy,"[counsellor, session, patient, therapy, sessio...","[As the patient, I recently had a conversation..."
4,3,154,3_school_grades_homework_class,"[school, grades, homework, class, exam, semest...","[I want to get a good grade on my final exam, ..."
...,...,...,...,...,...
229,228,11,228_add problems_experiences year_ago car_expe...,"[add problems, experiences year, ago car, expe...",[I have been diagnosed with posttraumatic stre...
230,229,11,229_hurtful_dwells_sorry resolved_argue ends,"[hurtful, dwells, sorry resolved, argue ends, ...",[We’ve been together almost three years. We ar...
231,230,11,230_make eye_eye contact_personalized haven_pu...,"[make eye, eye contact, personalized haven, pu...","[I've been suppressing it for quite some time,..."
232,231,10,231_just likely_likely helped_helped_likely,"[just likely, likely helped, helped, likely, p...",[I've always believed that no one would help m...


In [ ]:
# Initialize CountVectorizer with stop words removal (1 or 2 words)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=50)
dim_model = PCA(n_components=10)

vectorizer_model = CountVectorizer(stop_words='english', ngram_range=(1, 2))
# sentence_model = SentenceTransformer("all-MiniLM-L12-v2")
topic_model = BERTopic(hdbscan_model=cluster_model,n_gram_range=(1, 2), vectorizer_model=vectorizer_model, umap_model=dim_model)
topics, probs = topic_model.fit_transform(merged_df['Context'])

topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,340,0_didnt_just_went_remember,"[didnt, just, went, remember, really, time, go...","[I am an international student, and it is my f..."
1,1,336,1_good_think_patient_like,"[good, think, patient, like, im, doing, feel, ...",[I have been listening to others and learning ...
2,2,333,2_relationship_love_says_years,"[relationship, love, says, years, cheated, hes...",[My fiancé and I have been together for 3 year...
3,3,317,3_im_life_like_feel,"[im, life, like, feel, feel like, things, tryi...",[I'm always told I'm not good enough or trying...
4,4,316,4_emotions_feel_feeling_like,"[emotions, feel, feeling, like, feelings, ive,...",[I don't know how else to explain it. All I ca...
5,5,310,5_parents_mom_family_children,"[parents, mom, family, children, mother, schoo...","[I'm more like my dad, but as I get older I'm ..."
6,6,300,6_work_school_time_im,"[work, school, time, im, homework, like, day, ...",[I am on the track and softball team. My schoo...
7,7,299,7_overwhelmed_stress_feeling overwhelmed_work,"[overwhelmed, stress, feeling overwhelmed, wor...",[I am in a high stress position for a tech com...
8,8,296,8_ive feeling_ive_feeling_lately ive,"[ive feeling, ive, feeling, lately ive, lately...",[I just took a job that requires me to travel ...
9,9,285,9_people_like_feel_think,"[people, like, feel, think, im, ive, feel like...","[You know, I realize that I can't confront eve..."


In [ ]:
topic_model.visualize_topics()


In [ ]:
topic_model.visualize_barchart(top_n_topics=20)
# psych8k dataset has the following topics:
# Addiction, Anger, Anxiety, Bereavement, Culture, Depression, Eating_Disorders, Family, Fear, Illness, LGBTQ, Pain, Perfectionism and Stress.

# counsel-chat dataset has the following topics:
# Addiction, Anger, Anxiety, Counseling-Fundamentals, Depression, Violence, Eating_Disorders,
# Family-Conflict (Family), Grief (Bereavement), Human-Sexuality, Intimacy, Legal-Regulatory, LGBTQ,
# Marriage, Parenting, Relationships, Self-Harm, Spirituality, Stress, Substance-Abuse, Trauma

# Count the number of tokens in Clean data

In [ ]:
# df['Token Count'] = df['Clean'].str.split().str.len()
merged_df['Context_Token_Count'] = merged_df['Context'].str.split().str.len()
merged_df['Clean_Context_Token_Count'] = merged_df['Clean_Context'].str.split().str.len()

merged_df.head()

,Context,Response,Clean_Context,Clean_Response,Context_Token_Count,Clean_Context_Token_Count
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",going thing feeling barely sleep nothing think...,everyone think worthless maybe need find new p...,60,24
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",going thing feeling barely sleep nothing think...,hello thank question seeking advice feeling wo...,60,24
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,going thing feeling barely sleep nothing think...,first thing suggest getting sleep need impact ...,60,24
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,going thing feeling barely sleep nothing think...,therapy essential feeling depressed worthless ...,60,24
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,going thing feeling barely sleep nothing think...,first want let know alone feeling always someo...,60,24


# Using KeyBERT to extract keywords from text

In [ ]:
!pip install keyphrase_vectorizers

In [ ]:
from keyphrase_vectorizers import KeyphraseCountVectorizer, KeyphraseTfidfVectorizer
import gensim.downloader as api

In [ ]:
sample_df = merged_df.sort_values(ascending=False, by='Token Count').head(n=20)
sample_df.shape
# sample_df = merged_df.head(n=10)
# COLUMN = 'Clean'
COLUMN = 'Clean_Context'
MODEL = 'all-MiniLM-L12-v2'
model = KeyBERT(model=MODEL)
model.max_seq_length = 512

model_start = now()
doc_embeddings, word_embeddings = model.extract_embeddings(docs=sample_df[COLUMN].values, stop_words="english", vectorizer=KeyphraseCountVectorizer())
print('embedding time: {}'.format(now() - model_start))
keywords = model.extract_keywords(docs=sample_df[COLUMN].values, stop_words="english", top_n=1,
                                  vectorizer=KeyphraseCountVectorizer(),
                                  doc_embeddings=doc_embeddings, word_embeddings=word_embeddings)

print('model time: {}'.format(now() - model_start))

KeyError: 'Token Count'

# Finding most important topics

In [ ]:
TOP_N = 26
RESOLVE = {'I am' : 'I', 'I am': 'am',
           'going': 'going',
           'through': 'through',
           'some things': 'some', 'some things': 'things',
            'with my': 'with', 'with my': 'my',
            'feelings and myself': 'feelings', 'feelings and myself': 'and myself'}

sample_df['Top Keyword'] = [item[0][0] if len(item) else 'unknown' for item in keywords]
sample_df['Tag'] = sample_df['Top Keyword'].apply(func=lambda x: x if x not in RESOLVE.keys() else RESOLVE[x])
# all of the keywords out of the top N get retagged as unknown
sample_df['Tag'] = sample_df['Tag'].apply(func=lambda x: 'unknown' if x not in sample_df['Tag'].value_counts()[:TOP_N].index.tolist() else x)
print("how much of the corpus did we tag?")
print(sample_df['Tag'].value_counts(normalize=True))
print(sorted(sample_df['Tag'].unique().tolist()))

In [ ]:
sample_df

# Whole dataset

In [ ]:
COLUMN = 'Clean_Context'
MODEL = 'all-MiniLM-L12-v2'
model = KeyBERT(model=MODEL)
model.max_seq_length = 512

model_start = now()
document_embeddings, word_embeddings = model.extract_embeddings(docs=merged_df[COLUMN].values,)
print('embedding time: {}'.format(now() - model_start))
keywords = model.extract_keywords(docs=merged_df[COLUMN].values, top_n=1)
print('model time: {}'.format(now() - model_start))

In [ ]:
TOP_N = 26
RESOLVE = {'I am' : 'I', 'I am': 'am',
           'going': 'going',
           'through': 'through',
           'some things': 'some', 'some things': 'things',
            'with my': 'with', 'with my': 'my',
            'feelings and myself': 'feelings', 'feelings and myself': 'and myself'}

merged_df['Top Keyword'] = [item[0][0] if len(item) else 'unknown' for item in keywords]
merged_df['Tag'] = merged_df['Top Keyword'].apply(func=lambda x: x if x not in RESOLVE.keys() else RESOLVE[x])
# all of the keywords out of the top N get retagged as unknown
merged_df['Tag'] = merged_df['Tag'].apply(func=lambda x: 'unknown' if x not in merged_df['Tag'].value_counts()[:TOP_N].index.tolist() else x)
print("how much of the corpus did we tag?")
print(merged_df['Tag'].value_counts(normalize=True))
print(sorted(merged_df['Tag'].unique().tolist()))

In [ ]:
merged_df

# Visualization

Word Cloud for Keyword Frequency

In [ ]:
all_keywords = ' '.join([keyword for keywords in data['keywords'] for keyword, _ in keywords])
wordcloud = WordCloud(stopwords='english', background_color='white').generate(all_keywords)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Keyword Word Cloud')
plt.show()

Bar Plot for Keyword Frequency

In [ ]:
keyword_counter = Counter(all_keywords.split())
most_common_keywords = keyword_counter.most_common(10)

keywords, counts = zip(*most_common_keywords)
plt.figure(figsize=(10, 5))
plt.bar(keywords, counts)
plt.xlabel('Keywords')
plt.ylabel('Frequency')
plt.title('Top 10 Keywords')
plt.show()

Topic Distribution

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(y='topic', data=data, order=data['topic'].value_counts().index)
plt.xlabel('Count')
plt.ylabel('Topic')
plt.title('Topic Distribution')
plt.show()

In [ ]:
topic_model.visualize_topics()